# 概要

---

JupyterHub は、Jupyter Notebook というノートブックと呼ばれる形式を実行しながら、結果を記録しデータ分析などを進めるツールにユーザ認証をつけた管理サーバです。ユーザごとに Jupyter Notebook サーバを起動・停止することができます。

また、チームで Jupyter Notebook 環境を利用する場合にサーバ環境を統一することができます。サーバ環境を統一することにより、各ユーザの Jupyter Notebook 環境がベアメタル上の異なるサーバで起動することで生じる環境の乱立を排除することが可能となります。

# 設計

---

JupyterHub は Ansible で構成を管理します。

## インストール可能なOS

JupyterHub をインストールできるOS（ディストリビューション）は、以下のいずれかです。

- CentOS 7
- Debian 9
- Arch linux

それぞれ必要なパッケージが異なるため以下の jupyterhub ロールの tasks フォルダ内で各OS依存の構築を行っています。

In [ ]:
!find roles/jupyterhub/tasks/ -name "*.yml" ! -name "main.yml"

## パッケージ管理

OS固有のパッケージ管理(yumやdebなど)と pip、npm などのプログラム言語固有のパッケージ管理ツールを併用しています。

OS固有のパッケージ管理は、パッケージ名が異なる場合が多々あるため、各OS依存の tasks 内に記述します。逆に pip や npm はOSに依存しないため `main.yml` に記述します。

pip や npm はユーザのローカル環境に配置することも可能ですが、ユーザ環境に依存することによる問題を避ける方針のため、システム環境に配置をします。

Jupyter の一部のモジュールが公式のパッケージではなく、有志が GitHub でメンテしているもの利用しています。その場合でも、可能な限りシステム環境にインストールします。


## booststrap.sh

JupyterHub でログインして Jupyter インスタンスを起動するときに `/etc/jupyter/bootstrap.sh` を実行します。このスクリプトは、ユーザ環境において必要なモジュールの有効化や、Gitの初期設定を行います。

Gitの初期設定には、ユーザ名とメールアドレスの設定、ipynb ファイル形式をGitにコミットするときに出力結果は除外する処理を行っています。

このスクリプトを実行結果は `bootstrap.log` に保存されるので、うまく動かない場合はこのログファイルを確認することができます。ファイルが鬱陶しいのであれば、削除しても全く問題ありません。

## startup

Notebook を開いたときに `~/.ipython/profile_default/startup/` フォルダに存在する Pythonスクリプト を実行させることができます。

In [ ]:
!ls -l ~/.ipython/profile_default/startup/

この startup/99-requires.py に、シェルスクリプトの exit-code が `!0` のときにエラーにする処理を組み込んでいます。この処理がないと、Notebook のコードセルの実行に失敗してもエラーにならず処理が継続してしまうので、消さないように注意してください。

万が一、消してしまっても Jupyterインスタンスを停止→起動でファイルを復活させることができます。

## goss

構築後も構成情報を確認するために goss をインストールします。goss を使うことで、サーバ単体の正常性確認が簡単に行えます。

# 構築手順

---

Ansible を使って JupyterHub サーバ構築します。

## 諸注意

- SELinux は無効にしてください。CentOS はデフォルトで有効になっているので注意してください。
- ファイヤーウォールに注意してください。標準は 8000 ポートになっているので、CentOS の場合デフォルト設定だと拒否されています。

## pingの確認

以下を実行して、JupyterHub と疎通があることを確認してください。

※このpingはネットワークの到達性を確認する ping コマンドではなく、Ansible で対象のサーバにログインできるかの確認を行います。

In [ ]:
# ping確認
!ansible -i hosts -b -u osboxes -m ping jupyterhub

実行結果に`pong`という文字があれば正常です。もし、エラーになる場合は、ssh の公開鍵やパスワードが適切に設定されているかを確認してください。

## Playbookの確認

以下を実行して JupyterHub の Playbook をサーバに適用できるかを確認します。

In [ ]:
# Playbook確認
!ansible-playbook -i hosts jupyterhub.yml --check

failed=0 であれば正常です。

## Playbookの適用

以下を実行して JupyterHub の Playbook を対処ホストに適用します。（このコマンドを実行すると、サーバの構成が変更される場合があります）

In [ ]:
# ★構成変更★ Playbook適用
!ansible-playbook -i hosts jupyterhub.yml

failed=0 であれば、構築が完了です。

# 正常性確認手順

---

## ノードチェック

以下を実行して、ノードチェックを実行するホストを選択します。

In [ ]:
# ノードチェック
hosts = !ansible -i hosts jupyterhub --list-hosts | sed 1d | tr -d ' '
hosts.insert(0, "")
@interact(ホスト名=hosts)
def on_change(ホスト名):
    if ホスト名:
        global host
        host = ホスト名
        !ansible -i hosts {host} -m goss -a "path=/etc/goss.yml"

Failed: 0 であれば正常です。

## 外部からのアクセス確認

以下を実行した結果のURLにアクセスして、JupyterHub の画面が表示されるかを確認します。

In [ ]:
# URI出力
hosts = !ansible -i hosts --list-hosts jupyterhub | sed 1d | tr -d ' '
for h in hosts:
    print("http://{}:8000".format(h))

以上